# Tune GPT2 to generate controlled sentiment reviews
> Optimise GPT2 to produce IMDB movie reviews with controlled sentiment using a BERT sentiment classifier for rewards.

**WARNING:** We often experienced loss spikes in this examples which caused model training to fail or slow down. There is a [GitHub issue](https://github.com/lvwerra/trl/issues/101) to track the issue.

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2-ctrl-training-setup.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


The experiment setup is very similar to the positive sentiment notebook. However, in this notebook we fine-tune GPT2 (small) to generate **controlled** movie reviews based on the IMDB dataset. The model gets the target sentiment and 5 tokens from a real review and is tasked to produce continuations with the targeted sentiment. The reward for the continuations is calculated with the logits of a BERT sentiment classifier. That reward is then used for PPO training.

## Setup experiment

### Import dependencies

In [1]:
# !pip install trl==0.11.4

In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
import random
import torch
import wandb
import time
import os
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from random import choices
import matplotlib.pyplot as plt

tqdm.pandas()

from datasets import load_dataset

from transformers import AutoTokenizer, pipeline

from trl import (
    PPOTrainer,
    PPOConfig,
    AutoModelForCausalLMWithValueHead,
    create_reference_model,
)

### Configuration

In [4]:
commit_pipe_kwargs = {"top_k": None, "function_to_apply": "none"}

config = PPOConfig(
    model_name="/root/autodl-tmp/models/llama-2-7b-bnb-4bit",
    steps=51200,
    learning_rate=1.41e-5,
    remove_unused_columns=False,
    log_with="wandb",
)

txt_in_len = 5
txt_out_len = 20
seed = 1

/root/miniconda3/lib/python3.10/site-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


In [5]:
np.random.seed(seed)

You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://huggingface.co/papers/1909.08593). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

## Load data and models

### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [6]:
gpt2_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
gpt2_ref_model = create_reference_model(gpt2_model)
gpt2_tokenizer = AutoTokenizer.from_pretrained(config.model_name)

gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

In [7]:
# 确保 eos_token_id 是正数
print("eos_token_id:", gpt2_tokenizer.eos_token_id)
print("pad_token_id:", gpt2_tokenizer.pad_token_id)

eos_token_id: 2
pad_token_id: 2


### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 500 characters long and take the first 1000 characters of each comment. The first filter we apply to avoid comments that are less than `txt_in_len` token long and the second to avoid tokenizing way more text than we actually need.

In [8]:
import pandas as pd
dataset = pd.read_csv('/root/autodl-tmp/CommitFit/dataset/Ghadhab/dataset.csv')
dataset

,user,repo,commit,labels,msgs,diffs,feature
0,ponsonio,RxJava,0531b8bff5c14d9504beefb4ad47f473e3a22932,Perfective,Change hasException to hasThrowable--,diff --git a/rxjava-core/src/main/java/rx/Noti...,"[1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,ponsonio,RxJava,0950c46beda335819928585f1262dfe1dca78a0b,Adaptive,Trying to extend the Scheduler interface accor...,diff --git a/rxjava-core/src/main/java/rx/Sche...,"[2, 44, 0, 0, 30, 0, 0, 1, 18, 0, 0, 0, 0, 0, ..."
2,ponsonio,RxJava,0f92fdd8e6422d5b79c610a7fd8409d222315a49,Adaptive,RunAsync method for outputting multiple values--,diff --git a/rxjava-contrib/rxjava-async-util/...,"[2, 53, 0, 0, 42, 0, 0, 1, 45, 1, 0, 0, 0, 0, ..."
3,ponsonio,RxJava,100f571c9a2835d5a30a55374b9be74c147e031f,Corrective,forEach with Action1 but not Observer--I re-re...,diff --git a/language-adaptors/rxjava-groovy/s...,"[1, 5, 122, 9, 10, 9, 4, 1, 5, 18, 2, 0, 0, 0,..."
4,ponsonio,RxJava,191f023cf5253ea90647bc091dcaf55ccdce81cc,Corrective,1.x: Fix Completable swallows- OnErrorNotImple...,diff --git a/src/main/java/rx/Completable.java...,"[1, 1, 0, 0, 0, 0, 0, 1, 21, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...,...
1776,jenkinsci,clearcase-plugin,51e9da224f80254476a7dc446bca817b505381d8,Perfective,Use a temporary file to decrease memory consum...,diff --git a/src/main/java/hudson/plugins/clea...,"[2, 12, 0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
1777,jexp,batch-import,609d6c4b1eea2c33d9fb950fcbb9ba9dc1f80fc3,Perfective,added a more memory efficient structure for st...,diff --git a/src/main/java/org/neo4j/batchimpo...,"[10, 159, 29, 35, 9, 2, 1, 5, 106, 0, 4, 8, 0,..."
1778,hdiv,hdiv,19b650c78a1c76f4fd90274d7f163f863c0d39e4,Perfective,Memory and performance optimizations,diff --git a/hdiv-config/src/main/java/org/hdi...,"[31, 302, 131, 140, 170, 89, 53, 7, 88, 14, 17..."
1779,casidiablo,persistence,d7bf95159df37a3d338ca267dddd3d26b38ec37c,Perfective,Now it is possible to specify the sqlite open ...,diff --git a/pom.xml b/pom.xml\nindex 394263b....,"[5, 57, 20, 9, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
dataset[['msgs','labels']][:1]['msgs'][0],dataset[['msgs','labels']][:1]['labels'][0]

('Change hasException to hasThrowable--', 'Perfective')

In [10]:
dataset[['msgs','labels']][:2]['msgs'][1],dataset[['msgs','labels']][:2]['labels'][1]

('Trying to extend the Scheduler interface according- to the comments at -19.--',
 'Adaptive')

In [11]:
dataset[['msgs','labels']][:3]['msgs'][2],dataset[['msgs','labels']][:3]['labels'][2]

('RunAsync method for outputting multiple values--', 'Adaptive')

In [12]:
from datasets import load_dataset
# 使用自己的CSV文件
dataset = load_dataset(
     "csv",
    data_files="/root/autodl-tmp/CommitFit/dataset/Ghadhab/dataset.csv",  # 替换为你的文件路径
    split="train"
)


In [13]:
# # create the dataset
# #
# dataset = load_dataset("/root/.cache/huggingface/datasets/stanfordnlp___imdb", split="train")
# dataset = dataset.rename_columns({"text": "review", "label": "sentiment"})
# # make sure the comments are are at least 500 and trim to 1000
# dataset = dataset.filter(lambda x: len(x["review"]) > 500, batched=False)
# dataset = dataset.map(lambda x: {"review": x["review"][:1000]}, batched=False)

# dataset

### Tokenize IMDB reviews

We tokenize all IMDB in advance to avoid tokenizing twice. In the first step we encode the queries and slice the first `txt_in_len` tokens. In a second step we decode these tokens back to text for later display.

In [14]:
dataset

Dataset({
    features: ['user', 'repo', 'commit', 'labels', 'msgs', 'diffs', 'feature'],
    num_rows: 1781
})

In [15]:
dataset = dataset.map(
    lambda x: {
        "input_ids": gpt2_tokenizer.encode("Please read the following code and write comments in natural language:\n" + x["diffs"][:500]+"\nComments:", return_tensors="pt")[
            0, :txt_in_len
        ]
    },
    batched=False,
    # disable_tqdm=False
)
dataset = dataset.map(
    lambda x: {"query": gpt2_tokenizer.decode(x["input_ids"])}, batched=False
)
dataset = dataset[:20480]

from datasets import Dataset

dataset = Dataset.from_dict(dataset)
dataset.set_format("pytorch")

In [16]:
dataset[3]["input_ids"]

tensor([   1, 3529, 1303,  278, 1494])

In [17]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [18]:
ppo_trainer = PPOTrainer(
    config, gpt2_model, gpt2_ref_model, gpt2_tokenizer, dataset, data_collator=collator
)

/root/miniconda3/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tongjiajun1992 (tongjiajun1992-china-university-of-mining-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [19]:
from transformers import AutoModelForSequenceClassification

In [20]:
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
else:
    device = ppo_trainer.accelerator.device
commit_pipe = pipeline(
    "text-classification", "/root/autodl-tmp/CommitFit/notebooks/my_awesome_model/checkpoint-468", device=device
)

Device set to use cuda:0


The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [21]:
text = "Change hasException to hasThrowable--"
output = commit_pipe(text, **commit_pipe_kwargs)
output

[{'label': 'Corrective', 'score': 4.27896785736084},
 {'label': 'Perfective', 'score': -1.7570891380310059},
 {'label': 'Adaptive', 'score': -1.8728036880493164}]

In [22]:
text = "Trying to extend the Scheduler interface according- to the comments at -19.--"
output = commit_pipe(text, **commit_pipe_kwargs)
output

[{'label': 'Adaptive', 'score': 3.8357880115509033},
 {'label': 'Corrective', 'score': -2.2604589462280273},
 {'label': 'Perfective', 'score': -2.3742270469665527}]

In [23]:
text = "RunAsync method for outputting multiple values--"
output = commit_pipe(text, **commit_pipe_kwargs)
output

[{'label': 'Adaptive', 'score': 3.4162325859069824},
 {'label': 'Corrective', 'score': -1.7412129640579224},
 {'label': 'Perfective', 'score': -2.257861375808716}]

The resulting reward signal:

In [24]:
def extract_pipe_output(outputs):
    logits_dicts = []
    for out in outputs:    # 每个样本
        logits = {element["label"]: element["score"] for element in out}
        logits_dicts.append(logits)
    return logits_dicts

In [25]:
output[1]["score"]

-1.7412129640579224

### Control token dict
We will append the control token at the beginning of each query to signal the model what the target sentiment is. Each control sequence consists of three tokens:

In [26]:
# import torch
# 定义控制标签
ctrl_str = ["[Adaptive]", "[Perfective]", "[Corrective]"]

# 自动选择设备（GPU 优先）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 把 control string 编码成 token，用于 prepend 到输入
ctrl_tokens = {
    s: gpt2_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)
    for s in ctrl_str
}


In [27]:
ctrl_tokens

{'[Adaptive]': tensor([    1,   518, 29909,  1388,   415,   573, 29962], device='cuda:0'),
 '[Perfective]': tensor([    1,   518,  5894,  3647,   573, 29962], device='cuda:0'),
 '[Corrective]': tensor([    1,   518, 12521,  1621,   573, 29962], device='cuda:0')}

### Reward function

In [28]:
label_map = {
    "[Adaptive]": "Adaptive",
    "[Perfective]": "Perfective",
    "[Corrective]": "Corrective"
}

In [29]:
def task_logit_to_reward(logits, tasks):
    """
    logits: List[Dict] 每个样本的分类结果
    tasks:  List[str]  目标类别 ['[Adaptive]', '[Perfective]', '[Corrective]']
    """
    rewards = []
    for logit, task in zip(logits, tasks):
        # print(logit)
        mapped_task = label_map[task]   # 把 [Adaptive] → Adaptive
        if mapped_task not in logit:
            raise ValueError(f"Unknown task {mapped_task}, must be one of {list(logit.keys())}")
        rewards.append(torch.tensor(logit[mapped_task]))
    return rewards


The following examples show the rewards for the cases where the classifier logit is 4, -4 and 0 for the three targets ['Adaptive','Perfective','Corrective']. The scaling is not perfect as it differs between neutral and the other two classes. This is something to further investigate in the future. Ideally, one would use the logit output for each class individually, but since there is no dedicated class for neutral this is a workaround.

In [30]:
print(ctrl_str)

['[Adaptive]', '[Perfective]', '[Corrective]']


In [31]:
logits = [
    {"Adaptive": 0.7, "Perfective": 0.2, "Corrective": 0.1},   # 样本1
    {"Adaptive": 0.1, "Perfective": 0.6, "Corrective": 0.3},   # 样本2
    {"Adaptive": 0.2, "Perfective": 0.1, "Corrective": 0.7},   # 样本3
]

In [32]:
task_logit_to_reward(logits, ctrl_str)

[tensor(0.7000), tensor(0.6000), tensor(0.7000)]

In [33]:
# task_logit_to_reward(torch.Tensor([-4, -4, -4]), ctrl_str)

In [34]:
# task_logit_to_reward(torch.Tensor([0, 0, 0]), ctrl_str)

### Generation settings

In [35]:
generation_kwargs = {
    "min_length": 1,   # -1 会报错，改成 1 或 0
    "top_k": 0,        # 设为 0 表示不启用 top-k
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": gpt2_tokenizer.eos_token_id,
    "max_new_tokens": txt_out_len,
    "eos_token_id": gpt2_tokenizer.eos_token_id,
}

## Optimize model

**Steps**

The training loop consists of the following steps:
1. Get a batch of queries and create random controls
2. Get the query responses from the policy
3. Join query and responses and tokenize for BERT analysis
4. Get sentiments for query/responses from BERT
5. Optimize policy with PPO using the (query, response, reward) triplet
6. Log all the training statistics

**Training time**

This step takes **~2h** on a P6000 GPU with the above specified settings.

In [36]:
for epoch in range(2):
    for batch in tqdm(ppo_trainer.dataloader):
        (
            logs,
            game_data,
        ) = (
            dict(),
            dict(),
        )

        #### prepend a random control token
        task_list = choices(ctrl_str, k=config.batch_size)
        # print(task_list)
        game_data["query"] = [t + q for t, q in zip(task_list, batch["query"])]
        query_tensors = [
            torch.cat((ctrl_tokens[t], input_ids))
            for t, input_ids in zip(task_list, batch["input_ids"])
        ]

        #### get response from gpt2
        response_tensors = []
        for query in query_tensors:
            response = ppo_trainer.generate(query, **generation_kwargs)
            response_tensors.append(response.squeeze()[-txt_out_len:])
            # print(response)
        game_data["response"] = [
            gpt2_tokenizer.decode(r.squeeze()) for r in response_tensors
        ]
        
        #### commit analysis
        texts = [r for q, r in zip(batch["query"], game_data["response"])]
        print(texts)
        logits = extract_pipe_output(commit_pipe(texts, **commit_pipe_kwargs))
        # print(task_list)
        rewards = task_logit_to_reward(logits, task_list)
        print(rewards)
        #### Run PPO training
        t = time.time()
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        print(stats)
        for cs in ctrl_str:
            key = "env/reward_" + cs.strip("[]")
            stats[key] = np.mean(
                [r.cpu().numpy() for r, t in zip(rewards, task_list) if t == cs]
            )
        ppo_trainer.log_stats(stats, game_data, rewards)

  0%|          | 0/13 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['announcement carefully. keinan\nThe following text is a corrected version of the announcement which was', 'carefully. програма\nАвтор: GnS_19\nЗаголовок', 'terms and conditions of use (the “Terms”) carefully. живело 2015', "before attempting to install the software. nobody will help you if you don't.\nThe original", 'carefully. Begriffe\n[Perfective] Please read the following carefully.\nPlease read the following', 'terms and conditions carefully before using this site. Hinweis: Bitte lesen Sie die folgenden Bedingungen', 'before using the site\nThe material on this site is for general information purposes only and does not const', 'terms and conditions of use carefully before using the web site:\n nobody can use the web site for', 'terms and conditions carefully. październik 28, 2014\nTerms and', 'carefully. nobody is perfect, but we try to be.\n[Perfective] Please read', "list carefully and contact the appropriate person as soon as possible. Einzeln, but I'm afraid I", 'terms and c

OutOfMemoryError: CUDA out of memory. Tried to allocate 486.00 MiB. GPU 0 has a total capacity of 31.47 GiB of which 362.62 MiB is free. Including non-PyTorch memory, this process has 31.11 GiB memory in use. Of the allocated memory 27.02 GiB is allocated by PyTorch, and 3.79 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

### Training progress
If you are tracking the training progress with Weights&Biases you should see a plot similar to the following:

<div style="text-align: center">
<img src='https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/gpt2-ctrl-training-stats.png' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reward mean and distribution evolution during training. </p>
</div>

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher initial coefficient.

## Model inspection

### Reward distribution
First, we can have a look at the reward distribution. Both the negative and positive rewards are clearly shifted to high rewards. The neutral rewards, however, are still centered around zero. There are a few possible explanations for this. There could be a bug in the code and the way the neutral rewards are calculated. Another problem could be that sentence sometimes start with a strong sentiment and it is hard for the model shift the sentiment towards neutral.

In [ ]:
for ctrl_s in ctrl_str:
    plt.hist(
        [r for r, t in zip(logs["env/reward_dist"], task_list) if t == ctrl_s],
        density=True,
        alpha=0.5,
        label=ctrl_s,
    )
plt.legend(loc="best")
plt.title("reward distribution")
plt.grid(True)
plt.show()

## Save model
Finally, we save the model to disk for later usage.

In [ ]:
gpt2_model.save_pretrained("gpt2-imdb-ctrl")
gpt2_tokenizer.save_pretrained("gpt2-imdb-ctrl")